In [1]:
!pip install tslearn


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 197.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 168.1 MB/s eta 0:00:00a 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 122] Disk quota exceeded

